In [43]:
import sys
sys.path.append('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/backend/chatbot_utils')

import context_model as cm
import chatbot_utils as chatbot
import importlib
importlib.reload(cm)
importlib.reload(chatbot)

import pandas as pd
import hashlib
import json

with open('C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/openAI_api.json') as f:
    key = json.load(f)

DATA_DIR = 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset'


### Testing Embedding and Context Models

In [156]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()

In [157]:
cm.convert_logs_to_embeddings(f"{DATA_DIR}/final_log.out")

In [13]:
returns = cm.get_context("When was run sfa_mmi executed?", f"./chromadb/{md5hash}_large")
returns

([Document(page_content='Nov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:63]: sri_recall_log_lookup_location: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.verify_args:69]: working_dir: /home/instrument/fw/log/2023-11-09T13-11-48Z/sfa/NetworkEmulation1/sri\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,294+0000] [srss.__init__:43]: SRSS version: 05.01.01\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [utilities.register_in_persistence_registry_service:215]: Successfully registered in PersistenceRegistryService\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27299]: [INFO] [2023-11-09T13:15:16,306+0000] [srss.__init__:51]: Service registered\nNov 09 13:15:16 CMX50070-101776 start_mrt.sh[27255]: PyInstaller _MEI directory save_rec

### ChatGPT test

In [29]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()


In [101]:
chatbot.chat_with_bot("What?", md5hash, '002', memory=False)

{'answer': "I'm sorry, but the provided context doesn't contain any information about an error with _ifXTable_container_row_restore in a file.",
 'Source': ['audit(1699595347.228:252): proctitle=2F7573722F7362696E2F69707461626C6573002D2D77616974002D74006E6174002D4400444F434B4552002D7000746370002D6400302F30002D2D64706F72740034393939002D6A00444E4154002D2D746F2D64657374696E6174696F6E003130302E38322E36342E3133323A343939390000002D690062722D653937343130',
  'audit(1699595350.038:269): proctitle=2F7573722F7362696E2F69707461626C6573002D2D77616974002D740066696C746572002D4400444F434B45520000002D690062722D653937343130663832613539002D6F0062722D653937343130663832613539002D7000746370002D64003130302E38322E36342E313330002D2D64706F72740034393937002D6A004143',
  'audit(1699595350.035:268): proctitle=2F7573722F7362696E2F69707461626C6573002D2D77616974002D74006E6174002D4400444F434B4552002D7000746370002D6400302F30002D2D64706F72740034393937002D6A00444E4154002D2D746F2D64657374696E6174696F6E003130302E38322E363

In [102]:
chatbot.chat_with_bot("What SNMP packet(s) were recieved from UDP?", md5hash, '002', memory=True)

{'answer': 'The SNMP packets were received from UDP: [10.0.2.217]:41179->[10.102.37.173]:161.',
 'Source': ['Nov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[7] << R&S,1201.0002k70/101776,5.0.60.4\\n\nNov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[7] >> DIAG:UPD:TRAN:VERS?\\n\nNov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[7] << 281,152\\n',
  'Nov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] << R&S,1201.0002k70/101776,5.0.60.4\\n\nNov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] >> DIAG:UPD:TRAN:VERS?\\n\nNov 09 13:41:18 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] << 281,152\\n',
  'Nov 09 13:41:12 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] << R&S,1201.0002k70/101776,5.0.60.4\\n\nNov 09 13:41:12 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] >> DIAG:UPD:TRAN:VERS?\\n\nNov 09 13:41:12 CMX50070-101776 rspias-daemon[887]: PIAS: Socket[4] << 281,152\\n',
  'Nov 09 16:35:09 CMX50

In [38]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory

import os
os.environ["OPENAI_API_KEY"] = key['openai_api_key']

In [39]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, input_key='question', output_key='answer')

In [40]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()
question = "What lines have messages on SSH?"

In [41]:
docs_small,conf_small = cm.get_context(question, f"./chromadb/{md5hash}_small")
docs_large,conf_large = cm.get_context(question, f"./chromadb/{md5hash}_large")

## Add both large and small to a single list and sort them by confidence
docs = docs_small + docs_large
conf = conf_small + conf_large

df = pd.DataFrame({'docs':docs, 'conf':conf})
df.sort_values(by='conf', ascending=False, inplace=True)
df.reset_index(drop=True, inplace=True)

# df



In [22]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

vectorstore_relevant = Chroma.from_documents(list(df['docs'].values), embedding=embedding)
retriver = vectorstore_relevant.as_retriever(search_kwargs={"k": 10,"score_threshold": .5})


In [23]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
max_tokens=1000,
)

In [24]:
chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True, max_tokens_limit=1000, reduce_k_below_max_tokens=True)

In [25]:
result = chat({"question": question})
result

{'question': 'What lines have messages on SSH?',
 'chat_history': [HumanMessage(content='What lines have messages on SSH?'),
  AIMessage(content='The following lines have messages on SSH: \n\n1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"\n2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"\n3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Connection closed by 172.25.11.232 port 36124"\n4. "Nov 09 13:55:08 CMX50070-101776 sshd[281144]: Connection closed by 192.168.85.173 port 60810"\n5. "Nov 09 13:53:34 CMX50070-101776 sshd[271352]: Connection closed by 172.24.11.231 port 33828"\n\n')],
 'answer': 'The following lines have messages on SSH: \n\n1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"\n2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"\n3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Conn

In [26]:
print(result['answer'])

The following lines have messages on SSH: 

1. "Nov 09 13:50:04 CMX50070-101776 sshd[249289]: Connection closed by 172.25.11.232 port 45426"
2. "Nov 09 13:44:11 CMX50070-101776 sshd[212198]: Connection closed by 172.25.232.145 port 56312"
3. "Nov 09 13:54:33 CMX50070-101776 sshd[277530]: Connection closed by 172.25.11.232 port 36124"
4. "Nov 09 13:55:08 CMX50070-101776 sshd[281144]: Connection closed by 192.168.85.173 port 60810"
5. "Nov 09 13:53:34 CMX50070-101776 sshd[271352]: Connection closed by 172.24.11.231 port 33828"




### Full Token Test

In [49]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True, input_key='question', output_key='answer')

In [50]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)

# Loading the datastore
vector_db = Chroma(persist_directory=f"./chromadb/{md5hash}_small", embedding_function=embedding)
retriver = vector_db.as_retriever()

In [51]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)

chat = RetrievalQAWithSourcesChain.from_llm(llm, retriever=retriver, memory=memory, return_source_documents=True)

In [53]:
result = chat({"question": f"What IPs were they from?"})
result

{'question': 'What IPs were they from?',
 'chat_history': [HumanMessage(content='Were there any SSH login errors?'),
  AIMessage(content='Yes, there were SSH login errors. These include authentication failures and connection closures by the remote host. \n'),
  HumanMessage(content='What IPs were they from?'),
  AIMessage(content='The IP they were from is 10.0.2.217.\n')],
 'answer': 'The IP they were from is 10.0.2.217.\n',
 'sources': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out',
 'source_documents': [Document(page_content='Nov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217\nNov 09 14:28:28 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0.2.217', metadata={'source': 'C:/Users/vishw/OneDrive/Desktop/Projects/daemon-dialoguers/Dataset/test_log1.out'}),
  Document(page_content='Nov 09 14:31:45 CMX50070-101776 rspias-daemon[887]: PIAS: PIAS: Remote access from client 10.0

### Langchain Summary Model

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, RetrievalQAWithSourcesChain
from langchain.memory import ConversationBufferMemory
import tiktoken

import os
os.environ["OPENAI_API_KEY"] = key['openai_api_key']

md5hash = hashlib.md5(open(f"{DATA_DIR}/final_log.out",'rb').read()).hexdigest()


In [78]:
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
import textwrap
from time import monotonic

prompt_template = """Write a concise summary of the following. Start with a small paragraph and then mention some key events as bullet points:


{text}

Paragraph:

Important points:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])


In [79]:
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4"
)

In [81]:
# loader = TextLoader(f"{DATA_DIR}/test_log1.out")
# documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
# documents = text_splitter.split_documents(documents)
question = 'What are the important processes and aspects in the log?'
docs_small,conf_small = cm.get_context(question, f"./chromadb/{md5hash}_small", k = 50)
# docs_large,conf_large = cm.get_context(question, f"./chromadb/{md5hash}_large", k = 30)

## Add both large and small to a single list and sort them by confidence
docs = docs_small

In [82]:
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary = chain.run(docs)

In [83]:
print(summary)

The text is a collection of system logs and configurations. It captures various system events, such as initialization of loggers, setting log levels, rotation of log files, and system configuration. Throughout the passage, system services are started and logs are compressed and rotated to manage storage usage. There are also instances of logging configurations being read and written to specific directories.

Key Events:
- Initialization of multiple loggers including "log.rat.rqm", "log.coreLoad", "log.fast.core.assert", etc. with different log levels such as "debug" and "warn".
- Rotation of log files (sysmond_main.1.nmon, sysmond_processes.1.nmon) to manage storage space.
- Compression of log files, indicating efficient usage of storage space for log data.
- Instances of service registration and successful start up of services such as EventLog.
- Reading and writing of logging configurations from and to specific directories.
- System configurations including setting up traceSapIpAddre

## Clustering attempt

In [167]:
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA

In [168]:
embedding = OpenAIEmbeddings(
openai_organization=key['openai_organization'],
openai_api_key = key['openai_api_key'],
model="text-embedding-ada-002",
max_retries=10,
)
vector_db = Chroma(persist_directory=f"./chromadb/{md5hash}_small", embedding_function=embedding)


In [169]:
embeddings = np.array(vector_db.get(include=['embeddings'])['embeddings'])

pca = PCA(n_components=10)
pca.fit(embeddings)

pca_embeddings = pca.transform(embeddings)

## DBSCAN
dbscan = DBSCAN(eps=0.1, min_samples=3)
dbscan.fit(pca_embeddings)

## Get all the core points and outliers
core_points = np.where(dbscan.core_sample_indices_)[0]
outliers = np.where(dbscan.labels_ == -1)[0]

## Get the unique labels
labels = np.unique(dbscan.labels_)
labels

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
      dtype=int64)

In [188]:
outlier_behaviour = list(np.array(vector_db.get(include=['documents'])['documents'])[outliers][:80])

In [191]:
from langchain.document_loaders.telegram import text_to_docs
outlier_docs = text_to_docs(outlier_behaviour)

In [201]:
prompt_template = """Write these annomalies in form of bullet points:


{text}

Anomalies:"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary = chain.run(outlier_docs)

In [202]:
print(summary)

- The "diskstats_common.go:111" collector parsed an exclusion flag for diskstats.
- The "rspias-daemon[1016]" did not check signatures during the installation process.
- The "start-mrt.sysmond.sh[1066]" SYSMOND_SNAP_PSARGS command had unusual parameters.
- The "Module aufs" was not found in the specified directory.
- The "cni plugin" was not initialized due to a failure to load cni config.
- The session for user "xuser" was closed unexpectedly.
- The connection to X server was lost.
- The "AlfManager" was not started explicitly.
- The "libGL error" occurred because the radeonsi_dri.so file was not found in the specified directory.
- The "CMX_AU_IOCTL_DMA_DL_FPGA" call was made in the kernel.
- The "RohdeSchwarz.InstallationManagerAbstractionLib.InstallationManagerHandlerFactory.IsSetup2" function had an error.
- The "Rohde & Schwarz Installation Service" was started.
- The "Socket: connect operation" failed with error 110.
- The "systemd-tmpfiles-clean.service" deactivated successfully

In [206]:
md5hash = hashlib.md5(open(f"{DATA_DIR}/test_log1.out",'rb').read()).hexdigest()
chatbot.find_log_anomalies(md5hash)

'- Interface name is longer than 15 characters.\n- Repeated log entries of \'/opt/rohde-schwarz/installation-service/script/ /var/tmp/rohde-schwarz/rs-installation-service/script/\'.\n- Configuration read from file \'/etc/rspias.conf\', but no details provided about the configuration.\n- The entry of \'Indices successfully merged!\' does not specify which indices were merged.\n- The entry of \'Publish update:\' does not provide any details about the update.\n- Multiple entries of \'Nov 09 13:11:46 CMX50070-101776 kernel: get_generic_syssync_link_status called\' suggests a potential issue with the system sync status.\n- The entry of \'at RohdeSchwarz.InstallationManagerAbstractionLib.InstallationManagerHandlerFactory.IsSetup2(IBaseDevice device)\' does not provide enough context or information.\n- The log \'Nov 09 13:11:49 CMX50070-101776 load-images.sh[3269]: cmxrep-liveview-app\' does not provide any information about the status or outcome of the load-images process.\n- \'Socket[3]: D

### Testing Daemon summarizer

In [88]:
from langchain.document_loaders.telegram import text_to_docs

importlib.reload(cm)
warnings, errors, outliers = cm.get_anomalies(f"{DATA_DIR}/test_log1.out")
warnings, errors, outliers = text_to_docs(warnings), text_to_docs(errors), text_to_docs(outliers)

In [95]:
# prompt_template = """Give a brief summary in form of bullet points:


# {text}

# Key Points:"""

prompt_template = """Give a brief summary and mention the key points in form of bullet points :


{text}
"""

prompt = PromptTemplate(template=prompt_template, input_variables=["text"])
llm = ChatOpenAI(
openai_organization="org-kfsNXpcw90CoawqSyD7Mw4CD",
model="gpt-4",
)
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
summary = chain.run(errors)

In [96]:
print(summary)

Summary:
The text refers to various logs, notifications, and status updates from a technical system, possibly related to a device or software. The updates cover various aspects like the initialization of a logger, setting of log levels, running of a microservice, ignoring of key shortcuts, testing results, SNMP packet reception, stopping of processes, remote access, errors related to reader exclusivity, and termination of connections.

Key Points:
- Logger from RohdeSchwarz.CMXreporting.Logging.Logger initialized
- LogLevel set to Info
- RohdeSchwarz.CMXreporting.TransportInterfaceAdapter.Program running as an instrument in a Unix container
- RohdeSchwarz.Mrt.Microservices.PositioningCatrP3Dr.Positioning.PositioningServiceHost running at a certain point
- Ignored key shortcut: telephone=$dates
- Cannot find a keycode for keysym
- X server shutting down
- Supported_tests_from_offers has test Result Name A0 multiple times
- Error finding row index in _ifXTable_container_row_restore
- Rec